In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

In [5]:
data = pd.read_csv(r'C:\Users\COMP\Hameedhullah\Practice\Supervised Learning\Decision Tree\Classification\ASPR_Treatments_Locator_20250124.csv')
data.head(5)

,Provider Name,Address 1,Address 2,City,State,Zip,Public Phone,Latitude,Longitude,Geopoint,...,Has Veklury,Has Oseltamivir Generic,Has Oseltamivir Suspension,Has Oseltamivir Tamiflu,Has Baloxavir,Has Zanamivir,Has Peramivir,Grantee Code,Is Flu,Is COVID-19
0,CVS PHARMACY #05246,110 HWY 98,NaN,PORT SAINT JOE,FL,32456,NaN,30.016430,-85.430510,POINT (-85.43051 30.01643),...,False,False,False,False,False,False,False,CV1,False,True
1,PUBLIX PHARMACY #0567,2517 SOUTH FEDERAL HIGHWAY,NaN,FT PIERCE,FL,34982,NaN,27.319630,-80.316317,POINT (-80.316317 27.31963),...,False,False,False,False,False,False,False,PX1,False,True
2,WALGREENS #4379,4320 26TH ST W,NaN,BRADENTON,FL,34205-3563,941-755-8596,27.462954,-82.588373,POINT (-82.588373 27.462954),...,False,True,False,False,False,False,False,WG1,True,True
3,PUBLIX PHARMACY #0894,6820 OKEECHOBEE BLVD,NaN,WEST PALM BEACH,FL,33411,NaN,26.705894,-80.150789,POINT (-80.150789 26.705894),...,False,False,False,False,False,False,False,PX1,False,True
4,PUBLIX PHARMACY #0019,731 DUVAL STATION RD SUITE 4,NaN,JACKSONVILLE,FL,32218,NaN,30.485316,-81.604614,POINT (-81.604614 30.485316),...,False,False,False,False,False,False,False,PX1,False,True


In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65848 entries, 0 to 65847
Data columns (total 34 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Provider Name                   65848 non-null  object 
 1   Address 1                       65847 non-null  object 
 2   Address 2                       7373 non-null   object 
 3   City                            65847 non-null  object 
 4   State                           65847 non-null  object 
 5   Zip                             65847 non-null  object 
 6   Public Phone                    20559 non-null  object 
 7   Latitude                        65848 non-null  float64
 8   Longitude                       65848 non-null  float64
 9   Geopoint                        65848 non-null  object 
 10  Last Report Date                65847 non-null  object 
 11  Is PAP Site                     65848 non-null  bool   
 12  Prescribing Services Available  

In [7]:
data.isna().sum().sort_values()

Provider Name                         0
Has Peramivir                         0
Has Zanamivir                         0
Has Baloxavir                         0
Has Oseltamivir Tamiflu               0
Has Oseltamivir Suspension            0
Has Oseltamivir Generic               0
Has Veklury                           0
Has USG Lagevrio                      0
Has Commercial Lagevrio               0
Has Lagevrio                          0
Has USG Paxlovid                      0
Has Commercial Paxlovid               0
Has Paxlovid                          0
Has Commercial Product                0
Is Flu                                0
Has USG Product                       0
Home Delivery                         0
Is ICATT Site                         0
Latitude                              0
Longitude                             0
Geopoint                              0
Is COVID-19                           0
Is PAP Site                           0
Zip                                   1


In [10]:
na_feature = [feature for feature in data.columns if data[feature].isna().sum()>1]

for feature in na_feature:
    print(f"{feature}:{data[feature].isna().mean()*100:.3f}%missing value")

Address 2:88.803%missing value
Public Phone:68.778%missing value
Prescribing Services Available:98.642%missing value
Appointment URL:99.250%missing value
Grantee Code:30.941%missing value


In [11]:
data.describe(include = 'all').T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
Provider Name,65848,58285,HEB PHARMACY,259,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Address 1,65847,63078,101 N MAIN ST,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Address 2,7373,2181,CVS Pharmacy,2114,NaN,NaN,NaN,NaN,NaN,NaN,NaN
City,65847,8952,BROOKLYN,845,NaN,NaN,NaN,NaN,NaN,NaN,NaN
State,65847,55,CA,6190,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Zip,65847,44390,11214,32,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Public Phone,20559,20502,559-646-6618,23,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Latitude,65848.0,NaN,NaN,NaN,36.997081,5.600009,13.475204,33.636309,37.997045,40.868882,71.297375
Longitude,65848.0,NaN,NaN,NaN,-89.941546,16.00997,-159.672639,-96.803057,-85.331567,-78.84433,145.728316
Geopoint,65848,63661,POINT (-104.87489 40.89497),9,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
# numerical feature

numerical_feature = [feature for feature in data.columns if data[feature].dtypes != 'object']
print("no. of numerical column: ",len(numerical_feature))
data[numerical_feature].head()

no. of numerical column:  22


,Latitude,Longitude,Is PAP Site,Home Delivery,Is ICATT Site,Has USG Product,Has Commercial Product,Has Paxlovid,Has Commercial Paxlovid,Has USG Paxlovid,...,Has USG Lagevrio,Has Veklury,Has Oseltamivir Generic,Has Oseltamivir Suspension,Has Oseltamivir Tamiflu,Has Baloxavir,Has Zanamivir,Has Peramivir,Is Flu,Is COVID-19
0,30.016430,-85.430510,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
1,27.319630,-80.316317,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
2,27.462954,-82.588373,True,False,False,False,True,False,False,False,...,False,False,True,False,False,False,False,False,True,True
3,26.705894,-80.150789,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
4,30.485316,-81.604614,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True


In [14]:
# Categorical Column

categorical_feature = [feature for feature in data.columns if data[feature].dtypes == "object"]
print("no. of categorical column: ",len(categorical_feature))
data[categorical_feature].head()

no. of categorical column:  12


,Provider Name,Address 1,Address 2,City,State,Zip,Public Phone,Geopoint,Last Report Date,Prescribing Services Available,Appointment URL,Grantee Code
0,CVS PHARMACY #05246,110 HWY 98,NaN,PORT SAINT JOE,FL,32456,NaN,POINT (-85.43051 30.01643),01/17/2025 12:00:00 AM,NaN,NaN,CV1
1,PUBLIX PHARMACY #0567,2517 SOUTH FEDERAL HIGHWAY,NaN,FT PIERCE,FL,34982,NaN,POINT (-80.316317 27.31963),01/17/2025 12:00:00 AM,NaN,NaN,PX1
2,WALGREENS #4379,4320 26TH ST W,NaN,BRADENTON,FL,34205-3563,941-755-8596,POINT (-82.588373 27.462954),01/17/2025 12:00:00 AM,NaN,NaN,WG1
3,PUBLIX PHARMACY #0894,6820 OKEECHOBEE BLVD,NaN,WEST PALM BEACH,FL,33411,NaN,POINT (-80.150789 26.705894),01/17/2025 12:00:00 AM,NaN,NaN,PX1
4,PUBLIX PHARMACY #0019,731 DUVAL STATION RD SUITE 4,NaN,JACKSONVILLE,FL,32218,NaN,POINT (-81.604614 30.485316),01/17/2025 12:00:00 AM,NaN,NaN,PX1


In [20]:
data.nunique()

Provider Name                     58285
Address 1                         63078
Address 2                          2181
City                               8952
State                                55
Zip                               44390
Public Phone                      20502
Latitude                          63338
Longitude                         63447
Geopoint                          63661
Last Report Date                    981
Is PAP Site                           2
Prescribing Services Available        2
Appointment URL                       1
Home Delivery                         2
Is ICATT Site                         2
Has USG Product                       2
Has Commercial Product                2
Has Paxlovid                          2
Has Commercial Paxlovid               2
Has USG Paxlovid                      2
Has Lagevrio                          2
Has Commercial Lagevrio               2
Has USG Lagevrio                      2
Has Veklury                           2
